# Setup AWS EventBridge To Trigger a Pipeline Execution with S3

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
import json
from botocore.exceptions import ClientError

sess   = sagemaker.Session()
bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
account_id = boto3.client('sts').get_caller_identity().get('Account')

![Attach AWSStepFunctionsFullAccess Policy](img/attach_policies_with_stepfunctions.png)

## Steps
1. Create S3 Buckets
2. Enable CloudTrail Logging
3. Get StepFunctions Pipeline
4. Create EventBridge Rule
5. Test Trigger

### 1. Create S3 Data Upload Bucket (watched) & S3 Bucket for CloudTrail logs

In [ ]:
watched_bucket = 'dsoaws-data-upload-{}'.format(account_id)
print(watched_bucket)

In [ ]:
!aws s3 mb s3://$watched_bucket

In [ ]:
!aws s3 ls $watched_bucket

In [ ]:
cloudtrail_bucket = 'cloudtrail-dsoaws-{}'.format(account_id)
print(cloudtrail_bucket)

In [ ]:
!aws s3 mb s3://$cloudtrail_bucket

In [ ]:
!aws s3 ls $cloudtrail_bucket

### Attach the following S3 policy to the Cloud Trail logging bucket created above ^^

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AWSCloudTrailAclCheck20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:GetBucketAcl",
            "Resource": "arn:aws:s3:::{}".format(cloudtrail_bucket)
        },
        {
            "Sid": "AWSCloudTrailWrite20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:PutObject",
            "Resource": "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
            "Condition": {
                "StringEquals": {
                    "s3:x-amz-acl": "bucket-owner-full-control"
                }
            }
        },
        {
            "Sid": "AWSCloudTrailHTTPSOnly20180329",
            "Effect": "Deny",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
                "arn:aws:s3:::{}".format(cloudtrail_bucket)
            ],
            "Condition": {
                "Bool": {
                    "aws:SecureTransport": "false"
                }
            }
        }
    ]
}

print(policy)

In [ ]:
policy_json = json.dumps(policy)

In [ ]:
with open("policy.json", "w") as outfile: 
    json.dump(policy, outfile)

In [ ]:
!cat policy.json

In [ ]:
!aws s3api put-bucket-policy --bucket $cloudtrail_bucket --policy file://policy.json

### Create CloudTrail and enable logging

In [ ]:
!aws cloudtrail create-trail --name dsoaws --s3-bucket-name $cloudtrail_bucket --is-multi-region-trail

In [ ]:
!aws cloudtrail start-logging --name dsoaws

In [ ]:
!aws cloudtrail get-trail-status --name dsoaws

# Do the following in the AWS Console

1) Select S3

![Select S3 Service](img/select-s3.png)

2) Find the S3 BUCKET

![Select S3 Bucket](img/select-s3-bucket.png)

3) Click on Properties => Object-Level Logging

![Select S3 Bucket Properties](img/select-s3-bucket-properties.png)

4) Specify the CloudTrail and Click Create

![Enable S3 Object Logging](img/enable-s3-object-logging.png)

## Create Custom EventBridge EventBus

In [ ]:
events = boto3.client('events')

In [ ]:
try:
    response = events.create_event_bus(Name='dsoaws')
    print(response)
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceAlreadyExistsException':
        print("You're good. Event Bus already exists.")
    else:
        print("Unexpected error: %s" % e)

In [ ]:
response = events.describe_event_bus(Name='dsoaws')
eventbus_arn = response['Arn']
print(eventbus_arn)

## Create Custom EventBridge Rule

In [ ]:
pattern = {
  "source": [
    "aws.s3"
  ],
  "detail-type": [
    "AWS API Call via CloudTrail"
  ],
  "detail": {
    "eventSource": [
      "s3.amazonaws.com"
    ],
    "eventName": [
      "PutObject"
    ],
    "requestParameters": {
      "bucketName": [
        "{}".format(watched_bucket)
      ]
    }
  }
}

print(pattern)

In [ ]:
pattern_json = json.dumps(pattern)

In [ ]:
response = events.put_rule(
    Name='S3-Trigger',
    EventPattern=pattern_json,
    State='ENABLED',
    Description='Triggers an event on S3 PUT',
    EventBusName='dsoaws'
)
print(response)

In [ ]:
rule_arn = response['RuleArn']
print(rule_arn)

# Add Target

## Create IAM Role

In [ ]:
iam = boto3.client('iam')

In [ ]:
iam_role_name_eventbridge = 'DSOAWS_EventBridge_Invoke_StepFunctions'

### Create AssumeRolePolicyDocument

In [ ]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "events.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [ ]:
try:
    iam_role_eventbridge = iam.create_role(
        RoleName=iam_role_name_eventbridge,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS EventBridge Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

### Get the Role ARN

In [ ]:
role = iam.get_role(RoleName=iam_role_name_eventbridge)
iam_role_eventbridge_arn = role['Role']['Arn']
print(iam_role_eventbridge_arn)

# Get the StepFunctions ARN

In [ ]:
stepfunctions_arn = '<STEPFUNCTION_ARN_TO_TRIGGER>'

### Define Eventbridge Policy

In [ ]:
eventbridge_sfn_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "states:StartExecution"
            ],
            "Resource": [
                "{}".format(stepfunctions_arn)
            ]
        }
    ]
}
print(eventbridge_sfn_policy)

### Create Policy Object

In [ ]:
try:
    policy_eventbridge_sfn = iam.create_policy(
      PolicyName='DSOAWS_EventBridgeInvokeStepFunction',
      PolicyDocument=json.dumps(eventbridge_sfn_policy)
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

### Get ARN

In [ ]:
policy_eventbridge_sfn_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_EventBridgeInvokeStepFunction'
print(policy_eventbridge_sfn_arn)

### Attach Policy To Role

In [ ]:
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_eventbridge_sfn_arn,
        RoleName=iam_role_name_eventbridge
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

## Create EventBridge Rule Target

In [ ]:
sfn = boto3.client('stepfunctions')

In [ ]:
# ID: Id962e3164-fd3f-41a0-bb3d-47a1280f76fe

In [ ]:
import uuid

target_id = str(uuid.uuid4())

response = events.put_targets(
    Rule='S3-Trigger',
    EventBusName='dsoaws',
    Targets=[
        {
            'Id': target_id,
            'Arn': stepfunctions_arn,
            'RoleArn': iam_role_eventbridge_arn,
        }
    ]
)

In [ ]:
print(response)

## Check Stepfunctions - number of invocations

In [ ]:
response = sfn.list_executions(
    stateMachineArn=stepfunctions_arn)
number = len(response['executions'])
print(number)

## Test: Upload a file into the S3 bucket

In [ ]:
!aws s3 cp  ./src/requirements.txt s3://$watched_bucket

In [ ]:
response = sfn.list_executions(
    stateMachineArn=stepfunctions_arn)
number = len(response['executions'])
print(number)